In [ ]:
!pip install mujoco
!pip install -q mediapy

%env MUJOCO_GL=egl

env: MUJOCO_GL=egl


In [ ]:
import mujoco
import mediapy as media
import numpy as np

from scipy.linalg import solve_discrete_are

In [ ]:
xml = """
<mujoco>
  <worldbody>
    <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
    <body>
      <joint name="prism" type="slide" axis="1 0 0"/>
      <geom type="box" size=".4 .2 .1" rgba="0 1 0 1"/>
      <body>
        <joint type="hinge" axis="0 1 0"/>
        <geom type="cylinder" size=".05" fromto="0 0 0 0 0 1" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>

  <actuator>
    <motor joint="prism"/>
  </actuator>
</mujoco>
"""

In [ ]:
# initialization
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

## rendering
framerate = 30
renderer = mujoco.Renderer(model, height=360, width=640)
camera = mujoco.MjvCamera()
camera.distance = 5

# LQR
Q = np.eye(4)
R = 1e-4 * np.eye(1)

A = np.empty((4,4))
B = np.empty((4,1))

mujoco.mjd_transitionFD(model, data, 1e-6, 1, A, B, None, None)

P = solve_discrete_are(A, B, Q, R)
K = np.linalg.solve(B.T @ P @ B + R, B.T @ P @ A)

# Simulation
data.qpos = np.array([1, 0])

frames = []
while data.time < 10:
  x = np.concatenate((data.qpos, data.qvel, data.act))
  data.ctrl = - K @ x
  data.qfrc_applied = np.random.uniform(-20,20,2)
  mujoco.mj_step(model, data)

  if len(frames) < data.time * framerate:
    renderer.update_scene(data, camera)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=framerate)

This browser does not support the video tag.
